# TensorBoard HyperParameters Logging

Source: https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams

In this notebook, we'll use TensorBoard to compare multiple training runs with different hyper-parameters settings. Let us start by adding the TB extension to jupyter and clearing the logs directory.

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

# Clear any logs from previous runs
!rm -rf ./tb_log/ 

Now let's import the required modules. We'll use the `hparams` plugin of Keras to explore hyper-parameters settings. This plugin has several other functionalities not touched in this example.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorboard.plugins.hparams import api as hp

##### Load the dataset

We'll use again Fashion MNIST for this example.

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0


##### Build the model

We'll use a function to build a different version of the model based on some hyperparameters. In particular, we'll use only two hyperparameters here for simplicity:
- The number of units in the first fully-connected layer of our NN (`hparams['num_units']`)
- The dropout rate of that same layer (`hparams['dropout']`)

In [ ]:
def try_hp_setting(hparams):
    model = keras.models.Sequential([
        keras.layers.Flatten(),
        keras.layers.Dense(hparams['num_units'], activation=tf.nn.relu),
        keras.layers.Dropout(hparams['dropout']),
        keras.layers.Dense(10, activation=tf.nn.softmax),
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

##### Train the model

We'll train the model multiple times for different settings of the two hyper-parameters. Each run will be identified by an index (`session_num`). The corresponding logs will be saved in a separate sub-folder for TensorBoard.

The Keras `hparams` plugin has a dedicated callback (`hp.KerasCallback`) which logs the values of the hyper-parameters passed as input associated with each run.

In [ ]:
session_num = 0

In [ ]:
# for num_units in [512, 1024]:
for num_units in [16, 64]:
  for dropout_rate in [0.1, 0.5]:
        hparams = {'num_units': num_units, 'dropout': dropout_rate}
        run_name = "run-%d" % session_num
        print('--- Starting trial: %s' % run_name)
        print(hparams)
    
        log_dir = 'tb_log/hparam_tuning/' + run_name
        tb_callback = tf.keras.callbacks.TensorBoard(log_dir)
        hp_callback = hp.KerasCallback(log_dir, hparams)
    
        model = try_hp_setting(hparams)
        model.fit(x_train, y_train, validation_split=0.1, epochs=10, callbacks=[tb_callback, hp_callback])
        session_num += 1

##### Start Tensorboard

Let's start tensorboard and see the results of this experiment. We can clearly inspect the scalar logs as seen in the previous notebook. However, the `HPARAMS` tab provides a more useful visualization.

In [ ]:
%tensorboard --logdir './tb_log'